In [1]:
from src.common.vector import *
from src.common.reference_frame import *
from src.common.time import *
from datetime import datetime

In [3]:
# Exemplo 4.1 - Curtis

from src.common.vector import *
from src.common.reference_frame import *

# Parâmetros de entrada:
r = Vector(-5368,-1784,3691)

# Cálculo:
(ra,dec) = radec_from_r(r)

# Resultados:
print(f'RA = {round(np.rad2deg(ra),1)}       DEC = {round(np.rad2deg(dec),2)}')

RA = 198.4       DEC = 33.12


In [2]:


date_1 = datetime(2004,5,12,14,45,30)
jd_1 = julian_day(date_1)
date_2 = datetime(1957,10,4,19,26,24)
jd_2 = julian_day(date_2)
print(f'JD_1 = {jd_1} dias\nJD_2 = {jd_2} dias')
print(f'Diferença = {jd_1 - jd_2} dias')


JD_1 = 2453138.1149305557 dias
JD_2 = 2436116.31 dias
Diferença = 17021.804930555634 dias


In [3]:
from src.common.vector import *
from src.common.time import *
from datetime import datetime

theta = sidereal_time(date=datetime(2004,3,3,4,30,0),longitude=139.8)
theta

8.593542535242534

In [2]:
#Exemplo 5.7 - Curtis

from src.common.vector import *
from src.common.reference_frame import *

#Cálculos
r = Vector(x=-5368,y=-1784,z=3691)
alt = 0
lat = 20
lon = 60
r_o = get_topocentric_origin(longitude = 60,latitude = 20,altitude = 0,theta_g=126.7, is_geocentric_latitude=False)
rho = r-r_o
ra,dec = radec_from_r(rho)

#Resultados
print(f'r = {round(r_o.x,0)}I + ({round(r_o.y,1)})J + ({round(r_o.z,0)})K')
print(f'RA = {round(np.rad2deg(ra),1)}°\nDEC = {round(np.rad2deg(dec),2)}°')

r = -5955.0I + (-699.5)J + (2168.0)K
RA = 298.4°
DEC = 51.01°


In [4]:
from src.common.vector import *
from src.common.reference_frame import *
from datetime import datetime

# Observatório da Aldoria
lat = -30.5262693 + 360
lon = -70.8531799
alt = 2180e-3
date = datetime(2023,4,25,9,50,32)
R = get_topocentric_origin(latitude=lat,longitude=lon,altitude=alt,date=date)

ra_object = np.deg2rad(204.6559)
dec_object = np.deg2rad(-45.703182)

r = Vector(x = 1e4*np.cos(dec_object)*np.cos(ra_object),y=1e4*np.cos(dec_object)*np.sin(ra_object),z = 1e4*np.sin(dec_object))

rho = r - R

ra,dec = radec_from_r(rho)


print(f'R = {round(R.x,0)}I + ({round(R.y,1)})J + ({round(R.z,0)})K')
print(f'RA = {round(np.rad2deg(ra),1)}°\nDEC = {round(np.rad2deg(dec),2)}°')

R = 1879.0I + (-5179.8)J + (-3206.0)K
RA = 164.6°
DEC = -24.85°


In [8]:
# Exemplo 5.11 - Curtis

from src.common.vector import *
from src.common.reference_frame import *
from src.common.constants import EARTH_GRAVITATIONAL_PARAMETER

import numpy.polynomial.polynomial as poly

#Parâmetros de entrada:
lat = 40
alt = 1
local_sidereal_times = [44.506,45,45.499]
RAs = [43.537,54.420,64.318]
DECs = [-8.7833,-12.074,-15.105]
times = [0,118.10,237.58]

#Origens dos referências topocêntricos
R1 = get_topocentric_origin(latitude=lat,altitude=alt,theta=local_sidereal_times[0],is_geocentric_latitude=False)
R2 = get_topocentric_origin(latitude=lat,altitude=alt,theta=local_sidereal_times[1],is_geocentric_latitude=False)
R3 = get_topocentric_origin(latitude=lat,altitude=alt,theta=local_sidereal_times[2],is_geocentric_latitude=False)

#Posições relativas (rhos)
versor_rho1 = versor_from_radec(ra = RAs[0], dec = DECs[0])
versor_rho2 = versor_from_radec(ra = RAs[1], dec = DECs[1])
versor_rho3 = versor_from_radec(ra = RAs[2], dec = DECs[2])

#Intervalos de tempo
tau_1 = times[0]-times[1]
tau_3 = times[2]-times[1]
tau = times[2]-times[0]

#Cálculo dos produtos vetoriais:
p1 = cross_product(versor_rho2,versor_rho3)
p2 = cross_product(versor_rho1,versor_rho3)
p3 = cross_product(versor_rho1,versor_rho2)

#Constante D0:
D0 = dot_product(versor_rho1,p1)

#Constantes Ds:
D11 = dot_product(R1,p1)
D12 = dot_product(R1,p2)
D13 = dot_product(R1,p3)
D21 = dot_product(R2,p1)
D22 = dot_product(R2,p2)
D23 = dot_product(R2,p3)
D31 = dot_product(R3,p1)
D32 = dot_product(R3,p2)
D33 = dot_product(R3,p3)

#Constantes A e B:
A = (-D12*(tau_3/tau) + D22 + D32*(tau_1/tau))/D0
B = (1/(6*D0))*(D12*(tau_3**2 - tau**2)*tau_3/tau + D32*(tau**2-tau_1**2)*tau_1/tau)

#Constante E:
E = dot_product(R2,versor_rho2)

#Coeficientes do polnômio (r2)^8 + a(r2)^6 + b(r2)^3 + c:
a = -(A**2 + 2*A*E + R2.magnitude**2)
b = -2*EARTH_GRAVITATIONAL_PARAMETER*B*(A+E)
c = -(EARTH_GRAVITATIONAL_PARAMETER*B)**2

#Raízes do polinomio:
coef = [c,0,0,b,0,0,a,0,1]
roots = poly.polyroots(coef)

#Identificação de r2 dentre as raízes:
condition = (np.isreal(roots)) & (roots > 0)
roots_filtered = roots[condition]
r2_star = np.real(roots_filtered[0])

#Cálculo dos rho's:
rho1_mod = ((6*(D31*tau_1/tau_3 + D21*tau/tau_3)*r2_star**3 + EARTH_GRAVITATIONAL_PARAMETER*D31*(tau**2-tau_1**2)*tau_1/tau_3)/(6*r2_star**3 + EARTH_GRAVITATIONAL_PARAMETER*(tau**2-tau_3**2))-D11)/D0
rho2_mod = A + EARTH_GRAVITATIONAL_PARAMETER*B/(r2_star**3)
rho3_mod = ((6*(D13*tau_3/tau_1 - D23*tau/tau_1)*r2_star**3 + EARTH_GRAVITATIONAL_PARAMETER*D13*(tau**2-tau_3**2)*tau_3/tau_1)/(6*r2_star**3 + EARTH_GRAVITATIONAL_PARAMETER*(tau**2-tau_1**2))-D33)/D0
rho1 = Vector(x = rho1_mod*versor_rho1.x, y = rho1_mod*versor_rho1.y, z = rho1_mod*versor_rho1.z)
rho2 = Vector(x = rho2_mod*versor_rho2.x, y = rho2_mod*versor_rho2.y, z = rho2_mod*versor_rho2.z)
rho3 = Vector(x = rho3_mod*versor_rho3.x, y = rho3_mod*versor_rho3.y, z = rho3_mod*versor_rho3.z)

#Cálculo dos r's:
r1 = R1 + rho1
r2 = R2 + rho2
r3 = R3 + rho3

#Coeficientes de Lagrange:
f1 = 1-0.5*EARTH_GRAVITATIONAL_PARAMETER*(tau_1**2)/(r2_star**3)
f3 = 1-0.5*EARTH_GRAVITATIONAL_PARAMETER*(tau_3**2)/(r2_star**3)
g1 = tau_1 - (1/6)*EARTH_GRAVITATIONAL_PARAMETER*(tau_1**3)/(r2_star**3)
g3 = tau_3 - (1/6)*EARTH_GRAVITATIONAL_PARAMETER*(tau_3**3)/(r2_star**3)


#Velocidade:
v2 = (r3*f1 - r1*f3)*((f1*g3-f3*g1)**(-1))


#Resultados:
print('Origens das observações:')
print(f'R1 = {round(R1.x,1)}I + ({round(R1.y,1)})J + ({round(R1.z,1)})K')
print(f'R2 = {round(R2.x,1)}I + ({round(R2.y,1)})J + ({round(R2.z,1)})K')
print(f'R3 = {round(R3.x,1)}I + ({round(R3.y,1)})J + ({round(R3.z,1)})K')
print('-----------------')
print('Versores rhos:')
print(f'versor_rho1 = {round(versor_rho1.x,5)}I + ({round(versor_rho1.y,5)})J + ({round(versor_rho1.z,5)})K')
print(f'versor_rho2 = {round(versor_rho2.x,5)}I + ({round(versor_rho2.y,5)})J + ({round(versor_rho2.z,5)})K')
print(f'versor_rho3 = {round(versor_rho3.x,5)}I + ({round(versor_rho3.y,5)})J + ({round(versor_rho3.z,5)})K')
print('-----------------')
print('Intervalos temporais:')
print(f'tau_1 = {tau_1}')
print(f'tau_3 = {tau_3}')
print(f'tau = {tau}')
print('-----------------')
print('Produtos vetoriais:')
print(f'p1 = {round(p1.x,6)}I + ({round(p1.y,6)})J + ({round(p1.z,6)})K')
print(f'p2 = {round(p2.x,6)}I + ({round(p2.y,6)})J + ({round(p2.z,6)})K')
print(f'p3 = {round(p3.x,6)}I + ({round(p3.y,6)})J + ({round(p3.z,6)})K')
print('-----------------')
print('Constantes (Ds):')
print(f'D0 = {D0}')
print(f'D11 = {D11}     D12 = {D12}     D13 = {D13}')
print(f'D21 = {D21}     D22 = {D22}     D23 = {D23}')
print(f'D31 = {D31}     D32 = {D32}     D33 = {D33}')
print('-----------------')
print('Constantes A e B:')
print(f'A = {A}')
print(f'B = {B}')
print('-----------------')
print('Constantes E e módulo ao quadrado de R2:')
print(f'E = {E}')
print(f'R_2^2 = {R2.magnitude**2}')
print('-----------------')
print('Coeficientes do polinômio (r2^8 + a(r2)^6 + b(r2)^3 + c = 0):')
print(f'a = {round(a/1e7,4)}e+7')
print(f'b = {b}')
print(f'c = {c}')
print('-----------------')
print('valor de r2:')
print(f'r2 = {round(r2_star,1)}')
print('-----------------')
print('Módulos de rhos:')
print(f'rho_1 = {round(rho1_mod,1)}')
print(f'rho_2 = {round(rho2_mod,1)}')
print(f'rho_3 = {round(rho3_mod,1)}')
print('-----------------')
print('Vetores posição no Ref. Inercial:')
print(f'r1 = {round(r1.x,1)}I + ({round(r1.y,1)})J + ({round(r1.z,1)})K')
print(f'r2 = {round(r2.x,1)}I + ({round(r2.y,1)})J + ({round(r2.z,1)})K')
print(f'r3 = {round(r3.x,1)}I + ({round(r3.y,1)})J + ({round(r3.z,1)})K')
print('-----------------')
print('Coeficientes de Lagrange:')
print(f'f1 = {round(f1,5)}      g1 = {round(g1,5)}')
print(f'f3 = {round(f3,5)}      g3 = {round(g3,5)}')
print('-----------------')
print('Velocidade (v2):')
print(f'v2 = {round(v2.x,4)}I + ({round(v2.y,4)})J + ({round(v2.z,4)})K')



Origens das observações:
R1 = 3489.8I + (3430.2)J + (4078.5)K
R2 = 3460.1I + (3460.1)J + (4078.5)K
R3 = 3429.9I + (3490.1)J + (4078.5)K
-----------------
Versores rhos:
versor_rho1 = 0.71643I + (0.68074)J + (-0.1527)K
versor_rho2 = 0.56897I + (0.79531)J + (-0.20917)K
versor_rho3 = 0.4184I + (0.87008)J + (-0.26059)K
-----------------
Intervalos temporais:
tau_1 = -118.1
tau_3 = 119.48000000000002
tau = 237.58
-----------------
Produtos vetoriais:
p1 = -0.025251I + (0.060747)J + (0.162285)K
p2 = -0.044536I + (0.122804)J + (0.338522)K
p3 = -0.020952I + (0.062979)J + (0.182462)K
-----------------
Constantes (Ds):
D0 = -0.0015178807153869954
D11 = 782.1342298696927     D12 = 1646.490450136214     D13 = 887.0871432593711
D21 = 784.7043631501966     D22 = 1651.4927060471437     D23 = 889.596431802398
D31 = 787.2912400554336     D32 = 1656.5251522481512     D33 = 892.1201428189596
-----------------
Constantes A e B:
A = -9.255331042798277
B = 7676363777.219512
-----------------
Constantes E e 

In [5]:
# Exemplo 5.11 - Curtis

from src.orbit.orbit_determination import *

#Parâmetros de entrada:
lat = 40
alt = 1
local_sidereal_times = [44.506,45,45.499]
RAs = [43.537,54.420,64.318]
DECs = [-8.7833,-12.074,-15.105]
times = [0,118.10,237.58]

# Cálculo do Exemplo 5.11
r2,v2,R,rho,[f1,g1,f3,g3],D0,D = r2v2_from_angles_only_gauss_core(
    latitude_geodetic=lat,
    altitude=alt,
    local_sidereal_times=local_sidereal_times,
    RAs=RAs,
    DECs=DECs,
    times=times)

# Resultados 5.11
r2_literatura = Vector(5659.1,6533.8,3270.1)
v2_literatura = Vector(-3.8800,5.1156,-2.2387)

print('------- Vetores de estado 5.11 --------')
print(f'v2 = {round(v2.x,4)}I + ({round(v2.y,4)})J + ({round(v2.z,4)})K')
print(f'r2 = {round(r2.x,1)}I + ({round(r2.y,1)})J + ({round(r2.z,1)})K')
print('\n------ Diferenças percentuais 5.11 -------')
print(f'r2: e_x = {round(100*abs((r2.x-r2_literatura.x)/r2_literatura.x),2)}% e_y = {round(100*abs((r2.y-r2_literatura.y)/r2_literatura.y),2)}% e_z = {round(100*abs((r2.z-r2_literatura.z)/r2_literatura.z),2)}%')
print(f'v2: e_x = {round(100*abs((v2.x-v2_literatura.x)/v2_literatura.x),2)}% e_y = {round(100*abs((v2.y-v2_literatura.y)/v2_literatura.y),2)}% e_z = {round(100*abs((v2.z-v2_literatura.z)/v2_literatura.z),2)}%')

------- Vetores de estado 5.11 --------
v2 = -3.8791I + (5.1197)J + (-2.2409)K
r2 = 5659.7I + (6534.8)J + (3269.9)K

------ Diferenças percentuais 5.11 -------
r2: e_x = 0.01% e_y = 0.01% e_z = 0.01%
v2: e_x = 0.02% e_y = 0.08% e_z = 0.1%


In [1]:
# Exemplo 4.3 do Curtis

from src.common.vector import *
from src.orbit.orbit_determination import *

# Parâmetros de entrada:
r = Vector(-6045,-3490,2500)
v = Vector(-3.457,6.618,2.533)

# Chamada da função:
h,i,omega,e,w,theta = classic_orbital_elements_from_rv(r,v)
a = semimajor_axis_from_he(h,e)

#Resultados:
print('---------- Parâmetros de Órbita   -----------')
print(f'h = {round(h,0)}')
print(f'i = {round(i,1)}')
print(f'omega = {round(omega,1)}')
print(f'e = {round(e,4)}')
print(f'w = {round(w,2)}')
print(f'theta = {round(theta,2)}')
print(f'a = {round(a,0)}')

---------- Parâmetros de Órbita   -----------
h = 58312.0
i = 153.2
omega = 255.3
e = 0.1712
w = 20.07
theta = 28.45
a = 8788.0


In [1]:
# Exemplo 3.6 - Curtis
from src.orbit.orbit import *

# Parâmetros de Entrada:
t = 3600
r0 = 1e4
vr = 3.0752
a = -19655

# Cálculos:
x = find_universal_anomaly(t,r0,vr,1/a)

# Resultado:
x_curtis = 128.51

print(f'x = {round(x,2)}')
print(f'e_x = {round(100*abs((x-x_curtis)/x_curtis),2)}%')

x = 128.51
e_x = 0.0%


In [2]:
# Exemplo 3.7 - Curtis

from src.orbit.orbit import *

# Parâmetros de entrada:
r0 = Vector(7000,-12124,0)
v0 = Vector(2.6679,4.6210,0)
t = 3600 # [s]

# Cálculo:
r,v = rv_from_r0v0(r0,v0,t)

# Resultados:
r_curtis = Vector(-3296.8,7413.9,0)
v_curtis = Vector(-8.2977,-0.96309,0)

print('\n------- Vetores de estado --------')
print(f'v2 = {round(v.x,4)}I + ({round(v.y,4)})J + ({round(v.z,4)})K')
print(f'r2 = {round(r.x,1)}I + ({round(r.y,1)})J + ({round(r.z,1)})K')
print('\n------ Diferenças percentuais -------')
print(f'r2: e_x = {round(100*abs((r.x-r_curtis.x)/r_curtis.x),2)}% e_y = {round(100*abs((r.y-r_curtis.y)/r_curtis.y),2)}%')
print(f'v2: e_x = {round(100*abs((v.x-v_curtis.x)/v_curtis.x),2)}% e_y = {round(100*abs((v.y-v_curtis.y)/v_curtis.y),2)}%')


------- Vetores de estado --------
v2 = -8.2976I + (-0.964)J + (-0.0)K
r2 = -3297.8I + (7413.4)J + (0.0)K

------ Diferenças percentuais -------
r2: e_x = 0.03% e_y = 0.01%
v2: e_x = 0.0% e_y = 0.1%


In [13]:
# Exemplo 5.12 - Curtis

from src.orbit.orbit_determination import *

#Parâmetros de entrada:
r2_old = Vector(5659.1,6533.8,3270.1)
v2_old = Vector(-3.8800,5.1156,-2.2387)
R = [
    Vector(3489.8,3430.2,4078.5),
    Vector(3460.1,3460.1,4078.5),
    Vector(3429.9,3490.1,4078.5),
    ]
rho = [
    Vector(3639.1*0.71643,3639.1*0.68074,3639.1*(-0.15270)),
    Vector(3864.8*0.56897,3864.8*0.79531,3864.8*(-0.20917)),
    Vector(4172.8*0.41841,4172.8*0.87007,4172.8*(-0.26059)),
]
f1,f3,g1,g3 = (0.99648,0.99640,-117.97,119.33)
D0 = -0.0015198
D = [
    [782.15,1646.5,887.10],
    [784.72,1651.5,889.60],
    [787.31,1656.6,892.13]
    ]
times = [0,118.10,237.58]

# Cálculo do Exemplo 5.12
r2,v2 = improve_r2v2_from_gauss_angles_only(r2_old,v2_old,R,rho,f1,g1,f3,g3,D0,D,times)
h,i,omega,e,w,theta = classic_orbital_elements_from_rv(r2,v2)
a = semimajor_axis_from_he(h,e)
h_old,i_old,omega_old,e_old,w_old,theta_old = classic_orbital_elements_from_rv(r2_old,v2_old)
a_old = semimajor_axis_from_he(h_old,e_old)

# Resultados 5.12
r2_curtis = Vector(5662.1,6538.0,3269.0)
v2_curtis = Vector(-3.8856,5.1214,-2.2433)
h_curtis,a_curtis,e_curtis,i_curtis,omega_curtis,w_curtis,theta_curtis = (62818,1e4,0.1,30,270,90,45.01)

print('\n------- Vetores de estado 5.12 --------')
print(f'v2 = {round(v2.x,4)}I + ({round(v2.y,4)})J + ({round(v2.z,4)})K')
print(f'r2 = {round(r2.x,1)}I + ({round(r2.y,1)})J + ({round(r2.z,1)})K')
print('\n------ Diferenças percentuais 5.12 -------')
print(f'r2: e_x = {round(100*abs((r2.x-r2_curtis.x)/r2_curtis.x),2)}% e_y = {round(100*abs((r2.y-r2_curtis.y)/r2_curtis.y),2)}% e_z = {round(100*abs((r2.z-r2_curtis.z)/r2_curtis.z),2)}%')
print(f'v2: e_x = {round(100*abs((v2.x-v2_curtis.x)/v2_curtis.x),2)}% e_y = {round(100*abs((v2.y-v2_curtis.y)/v2_curtis.y),2)}% e_z = {round(100*abs((v2.z-v2_curtis.z)/v2_curtis.z),2)}%')

print('\n---------- Parâmetros de Órbita -----------')
print(f'h = {round(h,0)}    h_old = {round(h_old,0)}    e = {round(abs((h-h_curtis)/h_curtis),4)}%  e_old = {round(abs((h_old-h_curtis)/h_curtis),4)}%')
print(f'a = {round(a,0)}    a_old = {round(a_old,0)}    e = {round(abs((a-a_curtis)/a_curtis),4)}%  e_old = {round(abs((a_old-a_curtis)/a_curtis),4)}%')
print(f'e = {round(e,4)}    e_old = {round(e_old,4)}    e = {round(abs((e-e_curtis)/e_curtis),4)}%  e_old = {round(abs((e_old-e_curtis)/e_curtis),4)}%')
print(f'i = {round(i,0)}    i_old = {round(i_old,0)}    e = {round(abs((i-i_curtis)/i_curtis),4)}%  e_old = {round(abs((i_old-i_curtis)/i_curtis),4)}%')
print(f'omega = {round(omega,0)}    omega_old = {round(omega_old,0)}    e = {round(abs((omega-omega_curtis)/omega_curtis),4)}%  e_old = {round(abs((omega_old-omega_curtis)/omega_curtis),4)}%')
print(f'w = {round(w,0)}    w_old = {round(w_old,0)}    e = {round(abs((w-w_curtis)/w_curtis),4)}%  e_old = {round(abs((w_old-w_curtis)/w_curtis),4)}%')
print(f'theta = {round(theta,2)}    theta_old = {round(theta_old,2)}    e = {round(abs((theta-theta_curtis)/theta_curtis),4)}%  e_old = {round(abs((theta_old-theta_curtis)/theta_curtis),4)}%')



------- Vetores de estado 5.12 --------
v2 = -3.8597I + (5.1389)J + (-2.2464)K
r2 = 5667.7I + (6545.9)J + (3266.9)K

------ Diferenças percentuais 5.12 -------
r2: e_x = 0.1% e_y = 0.12% e_z = 0.06%
v2: e_x = 0.67% e_y = 0.34% e_z = 0.14%

---------- Parâmetros de Órbita -----------
h = 62851.0    h_old = 62704.0    e = 0.0005%  e_old = 0.0018%
a = 10018.0    a_old = 9959.0    e = 0.0018%  e_old = 0.0041%
e = 0.1035    e_old = 0.0975    e = 0.0347%  e_old = 0.0246%
i = 30.0    i_old = 30.0    e = 0.0024%  e_old = 0.0001%
omega = 270.0    omega_old = 270.0    e = 0.0008%  e_old = 0.0001%
w = 88.0    w_old = 89.0    e = 0.0173%  e_old = 0.0154%
theta = 46.77    theta_old = 46.35    e = 0.0391%  e_old = 0.0298%


In [2]:
# Exemplo 5.11 + 5.12

from src.orbit.orbit_determination import *

#Parâmetros de entrada:
lat = 40
alt = 1
local_sidereal_times = [44.506,45,45.499]
RAs = [43.537,54.420,64.318]
DECs = [-8.7833,-12.074,-15.105]
times = [0,118.10,237.58]

# Cálculo do Exemplo 5.11
r2_old,v2_old,R,rho,[f1,g1,f3,g3],D0,D = r2v2_from_angles_only_gauss_core(
    latitude_geodetic=lat,
    altitude=alt,
    local_sidereal_times=local_sidereal_times,
    RAs=RAs,
    DECs=DECs,
    times=times)

# r2,v2 = improve_r2v2_from_gauss_angles_only(r2_old,v2_old,R,rho,f1,g1,f3,g3,D0,D,times)
r2,v2 = r2v2_from_angles_only_gauss(
    latitude_geodetic=lat,
    altitude=alt,
    local_sidereal_times=local_sidereal_times,
    RAs=RAs,
    DECs=DECs,
    times=times)
h,i,omega,e,w,theta = classic_orbital_elements_from_rv(r2,v2)
a = semimajor_axis_from_he(h,e)
h_old,i_old,omega_old,e_old,w_old,theta_old = classic_orbital_elements_from_rv(r2_old,v2_old)
a_old = semimajor_axis_from_he(h_old,e_old)

# Resultados 5.12
r2_curtis = Vector(5662.1,6538.0,3269.0)
v2_curtis = Vector(-3.8856,5.1214,-2.2433)
h_curtis,a_curtis,e_curtis,i_curtis,omega_curtis,w_curtis,theta_curtis = (62818,1e4,0.1,30,270,90,45.01)

print('\n------- Vetores de estado 5.12 --------')
print(f'v2 = {round(v2.x,4)}I + ({round(v2.y,4)})J + ({round(v2.z,4)})K')
print(f'r2 = {round(r2.x,1)}I + ({round(r2.y,1)})J + ({round(r2.z,1)})K')
print('\n------ Diferenças percentuais 5.12 -------')
print(f'r2: e_x = {round(100*abs((r2.x-r2_curtis.x)/r2_curtis.x),2)}% e_y = {round(100*abs((r2.y-r2_curtis.y)/r2_curtis.y),2)}% e_z = {round(100*abs((r2.z-r2_curtis.z)/r2_curtis.z),2)}%')
print(f'v2: e_x = {round(100*abs((v2.x-v2_curtis.x)/v2_curtis.x),2)}% e_y = {round(100*abs((v2.y-v2_curtis.y)/v2_curtis.y),2)}% e_z = {round(100*abs((v2.z-v2_curtis.z)/v2_curtis.z),2)}%')

print('\n---------- Parâmetros de Órbita -----------')
print(f'h = {round(h,0)}    h_old = {round(h_old,0)}    e = {round(abs((h-h_curtis)/h_curtis),4)}%  e_old = {round(abs((h_old-h_curtis)/h_curtis),4)}%')
print(f'a = {round(a,0)}    a_old = {round(a_old,0)}    e = {round(abs((a-a_curtis)/a_curtis),4)}%  e_old = {round(abs((a_old-a_curtis)/a_curtis),4)}%')
print(f'e = {round(e,4)}    e_old = {round(e_old,4)}    e = {round(abs((e-e_curtis)/e_curtis),4)}%  e_old = {round(abs((e_old-e_curtis)/e_curtis),4)}%')
print(f'i = {round(i,0)}    i_old = {round(i_old,0)}    e = {round(abs((i-i_curtis)/i_curtis),4)}%  e_old = {round(abs((i_old-i_curtis)/i_curtis),4)}%')
print(f'omega = {round(omega,0)}    omega_old = {round(omega_old,0)}    e = {round(abs((omega-omega_curtis)/omega_curtis),4)}%  e_old = {round(abs((omega_old-omega_curtis)/omega_curtis),4)}%')
print(f'w = {round(w,0)}    w_old = {round(w_old,0)}    e = {round(abs((w-w_curtis)/w_curtis),4)}%  e_old = {round(abs((w_old-w_curtis)/w_curtis),4)}%')
print(f'theta = {round(theta,2)}    theta_old = {round(theta_old,2)}    e = {round(abs((theta-theta_curtis)/theta_curtis),4)}%  e_old = {round(abs((theta_old-theta_curtis)/theta_curtis),4)}%')


------- Vetores de estado 5.12 --------
v2 = -3.8848I + (5.1254)J + (-2.2446)K
r2 = 5662.7I + (6539.0)J + (3268.8)K

------ Diferenças percentuais 5.12 -------
r2: e_x = 0.01% e_y = 0.01% e_z = 0.01%
v2: e_x = 0.02% e_y = 0.08% e_z = 0.06%

---------- Parâmetros de Órbita -----------
h = 62851.0    h_old = 62739.0    e = 0.0005%  e_old = 0.0013%
a = 10013.0    a_old = 9972.0    e = 0.0013%  e_old = 0.0028%
e = 0.1011    e_old = 0.0987    e = 0.0109%  e_old = 0.0135%
i = 30.0    i_old = 30.0    e = 0.0002%  e_old = 0.0005%
omega = 270.0    omega_old = 270.0    e = 0.0001%  e_old = 0.0%
w = 90.0    w_old = 89.0    e = 0.0022%  e_old = 0.0122%
theta = 44.82    theta_old = 46.09    e = 0.0042%  e_old = 0.024%
